Ever wanted to have multiple branches checked out in git?

Since July 2015 (https://github.com/blog/2042-git-2-5-including-multiple-worktrees-and-triangular-workflows), git worktrees was introduced to solve this problem. Let me show you how to use this with some examples. I will create a git repo called "foo":

<code>$ git init ~/foo</code>

When I do this, my git repo is now located at <code>~/foo/.git</code> If I view what's under the .git folder, I have the following files:

* HEAD
* config
* description

And the following folders:

* branches
* hooks
* info
* objects
* refs

I will add one file to master and commit it:

<code>$ touch a.txt; git add a.txt; git commit -m "First checkin"</code>

OK, now let's create a branch named "dev":

<code>$ git branch dev</code>

Now, let's create a folder <code>~/foo/worktrees</code> called "worktrees" and switch to it (the name of this folder doesn't matter)

<code>$ mkdir worktrees; cd worktrees</code>

From here, we now run the following git command:

<code>$ git worktree add dev-branch dev</code>

After this, you'll get some output like the following:

The git worktree takes the following form:

<code>git worktree add [name-of-our-worktree] [branch-name]</code>

 You'll see a new directory appear named "dev-branch". If you nagivate inside, you are now in the "dev" working directory! But you can back out, and be in the "master" working directory. So in git, we refer to the working directories created by git worktree as <b>linked working trees</b>. This is distinquished from our <b>main working tree</b> which in this case is master.

What happens if you try to create a linked working tree for master? Note that our main working tree is master. Well, we can run:

<code>$ git worktree add master master</code>

We get the following output:

<code>$ fatal: 'master' is already checked out at '~/foo'</code>

Git doesn't allow us to have two working directories that would be the same. However, if you added the -f flag, you can force git to do this anyway. I'm not aware of a use case where you would want to do this.

OK, another thing to notice, is that there is a file in our linked working tree named ".git". What is this file? If we look inside, we find:

<code>$gitdir: ~/foo/.git/worktrees/dev-branch</code>

If we now look back in our <code>~/foo/.git</code>, we find a new folder called "worktrees". Insider this folder, there is a folder for each linked working tree. If we look inside the "dev" folder, we see the following files:

* HEAD
* ORIG_HEAD
* gitdir
* commondir
* index

and folders:

* logs

If we look at contents of commondir, it is "../..". It's relative path, and this relative path is relative to GIT_DIR, which in this case is <code>~/foo/.git/worktrees/dev-branch</code>. Thus, GIT_COMMON_DIR points to <code>~/foo/.git</code>

# Viewing Worktrees

At any point, if you want to list all working trees (including the main working tree), run:

<code>$ git worktree list</code>

This yields (given our current setup):

<code>~/foo/.git           (bare)
~/foo/worktrees/dev-branch  9aaccf4 [dev]
</code>

Note the main working directory is included in this list

# Deleting Worktrees

To delete a linked working tree, simply delete the folder. When you do this, you'll notice that in the main git directory, the worktree information is still there. If you want to clean this up, run:

<code>git worktree prune</code>

Else, git will clean it up automatically based on the value set in your git config for <code>gc.worktreePruneExpire</code>

# Moving Worktrees

What happens if you move your working directory? Let's do this:

<code>$ mv dev-branch dev-branch-move</code>

Now, if we run <code>git worktree list</code> it still reports <code>~/foo/worktrees/dev-branch</code> as the path

To update, I modified the following items:

* <code>~/foo/worktrees/dev-branch-move/.git</code>
* <code>~/foo/.git/worktrees/dev-branch --> ~/foo/.git/worktrees/dev-branch-move</code>
* <code>~/foo/.git/worktrees/dev-branch-move/gitdir</code>

Also, I suppose you would need to update <code>~/foo/.git/worktrees/dev-branch-move/gitdir</code> if you moved the linked directory tree to a different directory

# What's next?
In https://git-scm.com/docs/git-worktree, it is addressed that <code>remove</code> and <code>mv</code> commands would be good for <code>git worktree</code>, and I agree. Right now, it's a set of manual steps.